# Connecting the Proper data to perform the neccary 

In [22]:
#Importing Dependencies
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.preprocessing import StandardScaler
import psycopg2

In [2]:
#establishing the connection
conn = psycopg2.connect(
   database="movies_db", user='postgres', password='finalproject', host='moviesdb.ct0o6q0pbonx.us-east-2.rds.amazonaws.com', port= '5432'
)

In [3]:
#Setting auto commit false
conn.autocommit = True


In [4]:
#Creating a cursor object using the cursor() method
cursor = conn.cursor()


In [5]:
cursor.execute('''select m.id, m.title , m.release_date, m.revenue
from movies_metadata m 
where m.status = 'Released' ''')

In [6]:
#Fetching 1st row from the table
result = cursor.fetchone();
print(result)


('862', 'Toy Story', datetime.date(1995, 10, 30), Decimal('373554033.0'))


In [7]:
#Fetching all rows from the table
result = cursor.fetchall();
print(result)


[('8844', 'Jumanji', datetime.date(1995, 12, 15), Decimal('262797249.0')), ('31357', 'Waiting to Exhale', datetime.date(1995, 12, 22), Decimal('81452156.0')), ('11862', 'Father of the Bride Part II', datetime.date(1995, 2, 10), Decimal('76578911.0')), ('949', 'Heat', datetime.date(1995, 12, 15), Decimal('187436818.0')), ('9091', 'Sudden Death', datetime.date(1995, 12, 22), Decimal('64350171.0')), ('710', 'GoldenEye', datetime.date(1995, 11, 16), Decimal('352194034.0')), ('9087', 'The American President', datetime.date(1995, 11, 17), Decimal('107879496.0')), ('21032', 'Balto', datetime.date(1995, 12, 22), Decimal('11348324.0')), ('10858', 'Nixon', datetime.date(1995, 12, 22), Decimal('13681765.0')), ('1408', 'Cutthroat Island', datetime.date(1995, 12, 22), Decimal('10017322.0')), ('524', 'Casino', datetime.date(1995, 11, 22), Decimal('116112375.0')), ('4584', 'Sense and Sensibility', datetime.date(1995, 12, 13), Decimal('135000000.0')), ('5', 'Four Rooms', datetime.date(1995, 12, 9), De

In [8]:
data = pd.DataFrame(result,columns=["id","Title","Release Date","Revenue"])
data.head()

,id,Title,Release Date,Revenue
0,8844,Jumanji,1995-12-15,262797249.0
1,31357,Waiting to Exhale,1995-12-22,81452156.0
2,11862,Father of the Bride Part II,1995-02-10,76578911.0
3,949,Heat,1995-12-15,187436818.0
4,9091,Sudden Death,1995-12-22,64350171.0


In [9]:
len(data)

7384

In [10]:
data["Revenue"] = data["Revenue"].astype('int64')


In [11]:
data['Release Date'] = pd.to_datetime(data['Release Date'])

In [12]:
data = data.dropna(subset=['Revenue'])

In [13]:
data.head()

,id,Title,Release Date,Revenue
0,8844,Jumanji,1995-12-15,262797249
1,31357,Waiting to Exhale,1995-12-22,81452156
2,11862,Father of the Bride Part II,1995-02-10,76578911
3,949,Heat,1995-12-15,187436818
4,9091,Sudden Death,1995-12-22,64350171


In [14]:
len(data)

7384

In [15]:
#Closing the connection
conn.close()

# Markdown

In [16]:
#Printing out the dataframe columns
data.columns

Index(['id', 'Title', 'Release Date', 'Revenue'], dtype='object')

In [17]:
#These two variables are what the machine learning will be testing
# X= revenue and y= release_date
target=["Release Date"]

In [18]:
#Showing the dateframe again
data.head()

,id,Title,Release Date,Revenue
0,8844,Jumanji,1995-12-15,262797249
1,31357,Waiting to Exhale,1995-12-22,81452156
2,11862,Father of the Bride Part II,1995-02-10,76578911
3,949,Heat,1995-12-15,187436818
4,9091,Sudden Death,1995-12-22,64350171


In [29]:
# Create our features
X = data.copy()
X = data.drop(columns=['Release Date', 'id','Title'])

# Create our target
y = data.loc[:, target].copy()

In [30]:
#Showing the statics of the dataframe
X.describe()

,Revenue
count,7.384000e+03
mean,6.892740e+07
std,1.465760e+08
min,1.000000e+00
25%,2.405036e+06
50%,1.686813e+07
75%,6.763460e+07
max,2.787965e+09


In [31]:
#Setting up the testing model for our machine learning 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [32]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5538, 1)
(1846, 1)
(5538, 1)
(1846, 1)


In [33]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [34]:
# Determine the shape of our training and testing sets.
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(5907, 1)
(1477, 1)
(5907, 1)
(1477, 1)


In [35]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [37]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [41]:
predictions

array(['1981-06-26T00:00:00.000000000', '1985-02-15T00:00:00.000000000',
       '2008-05-14T00:00:00.000000000', ...,
       '2007-10-26T00:00:00.000000000', '1992-11-05T00:00:00.000000000',
       '1955-10-03T00:00:00.000000000'], dtype='datetime64[ns]')

In [42]:
model

DecisionTreeClassifier()

In [40]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=[ "Predicted 0","Predicted 1"])

cm_df

ValueError: Shape of passed values is (2603, 2603), indices imply (2, 2)

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))